# 1. 필요한 라이브러리 설치 및 import

In [1]:
!pip install -U transformers
!pip install -U datasets
!pip install -U bitsandbytes
!pip install -U peft
!pip install -U trl
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 92.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from datasets import Dataset, load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
from huggingface_hub import notebook_login
notebook_login()

# 2. 데이터셋 로드

In [4]:
jsonl_file = "/content/drive/MyDrive/Colab Notebooks/기타 공부/구글 머신러닝 부트캠프/Gemma/cleaned_labeldata.jsonl"

dataset = load_dataset('json', data_files=jsonl_file)

Generating train split: 0 examples [00:00, ? examples/s]

# 3. 한국어 요약 모델

<Gemma base 모델과 IT 모델의 차이점>
- base 모델은 튜닝 작업을 거치지 않은 원본의 모델로써, 다른 방식으로 Instruct tuning을 거쳐 사용자의 목적에 더 적합한 모델을 만들 수 있음
- IT 모델이 대화 형식의 상호작용에 최적화되어 사용자의 의도를 더 정확하게 파악하여 보다 정제된 답변을 제공 (ChatGPT와 유사)

## Gemma-it

In [5]:
BASE_MODEL = "google/gemma-2-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [6]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['context'])):
        prompt_list.append(r"""<bos><start_of_turn>user
이 사람과 가장 닮은 동물의 이름만 출력해 주세요 :

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['context'][i], example['animal'][i]))
    return prompt_list

In [7]:
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
이 사람과 가장 닮은 동물의 이름만 출력해 주세요 :

나는 남자다. 나는 60대 초반이다. 나는 작은 글씨 보기를 꺼려 한다. 나는 동물복지에 관심이 많다. 나는 지난주에 새 보험에 가입했다.<end_of_turn>
<start_of_turn>model
코알라<end_of_turn><eos>


In [8]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [9]:
BASE_MODEL = "google/gemma-2-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs=10,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        save_steps=25,
        logging_steps=25,
        learning_rate=2e-4,
        weight_decay=0.001,
        fp16=False,
        bf16=False,
        max_grad_norm=0.3,
        max_steps=-1,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="constant",
        report_to="tensorboard"
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [11]:
trainer.train()

Step,Training Loss
25,2.153200
50,1.214000
75,1.136300
100,0.858800
125,1.013100
150,0.788900
175,0.860600
200,0.743200
225,0.856800
250,0.727400


TrainOutput(global_step=6870, training_loss=0.2365551645231733, metrics={'train_runtime': 3046.1677, 'train_samples_per_second': 9.021, 'train_steps_per_second': 2.255, 'total_flos': 2.626868548942848e+16, 'train_loss': 0.2365551645231733, 'epoch': 10.0})

In [12]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [13]:
!ls -alh lora_adapter

total 30M
drwxr-xr-x 2 root root 4.0K Oct  3 01:49 .
drwxr-xr-x 1 root root 4.0K Oct  3 01:49 ..
-rw-r--r-- 1 root root  721 Oct  3 01:49 adapter_config.json
-rw-r--r-- 1 root root  30M Oct  3 01:49 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Oct  3 01:49 README.md


In [14]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2-2b-it-petrsona')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
!ls -alh ./gemma-2-2b-it-petrsona

total 4.9G
drwxr-xr-x 2 root root 4.0K Oct  3 01:50 .
drwxr-xr-x 1 root root 4.0K Oct  3 01:49 ..
-rw-r--r-- 1 root root  880 Oct  3 01:49 config.json
-rw-r--r-- 1 root root  187 Oct  3 01:49 generation_config.json
-rw-r--r-- 1 root root 4.7G Oct  3 01:50 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root 230M Oct  3 01:50 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root  24K Oct  3 01:50 model.safetensors.index.json


In [16]:
model.push_to_hub("hyesszx/Petrsona")
tokenizer.push_to_hub("hyesszx/Petrsona")

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/hyesszx/Petrsona/commit/3575824938a6509e7a2ea23fe4bd66f7920582c9', commit_message='Upload tokenizer', commit_description='', oid='3575824938a6509e7a2ea23fe4bd66f7920582c9', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
BASE_MODEL = "google/gemma-2-2b-it"
FINETUNE_MODEL = "./gemma-2-2b-it-petrsona"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
finetune_model.push_to_hub("hyesszx/Petrsona")
tokenizer.push_to_hub("hyesszx/Petrsona")

In [18]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

In [19]:
messages = [
    {
        "role": "user",
        "content": "이 사람과 가장 닮은 동물의 이름만 출력해 주세요 :\n\n 나는 여자다. 나는 대학생이다. 나는 조용한 성격이다. 나는 인공지능학과이다. 나는 운동을 좋아한다."
    }
]
prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [20]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


여우
